In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import statistics
import networkx as nx
import warnings
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
df = pd.read_csv('./data/2021.01.19 spotify_us_all_fe.csv')
df = df[df['No. of Artists'] > 1]
df = df[list(df.columns[1:11]) + ['Song_days_onchart', 'Profit']]
df.head()

,Date,Track Name,Streams,Position,Artist Name,Artist No.,No. of Artists,Artist Genre,Track URI,Country,Song_days_onchart,Profit
0,2017-01-01,Lighthouse - Andrelli Remix,149929.0,183.0,Hearts & Colors,1,2,['viral pop'],04CttTezSnv71USiiG9mIo,United States,1,496.26499
1,2017-01-01,Lighthouse - Andrelli Remix,149929.0,183.0,Andrelli,2,2,['swedish pop'],04CttTezSnv71USiiG9mIo,United States,1,496.26499
2,2017-01-01,In the Name of Love,435945.0,27.0,Martin Garrix,1,2,"['big room', 'dance pop', 'edm', 'electro hous...",04DwTuZ2VBdJCCC5TROn7L,United States,1,1442.97795
3,2017-01-01,In the Name of Love,435945.0,27.0,Bebe Rexha,2,2,"['dance pop', 'electropop', 'pop', 'pop dance'...",04DwTuZ2VBdJCCC5TROn7L,United States,1,1442.97795
4,2017-01-01,Party,151474.0,178.0,Chris Brown,1,3,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap']",05Z7jet4VDNVgNQWcYHnrk,United States,1,501.37894


In [5]:
df['Sub-genres'] = df['Artist Genre']
df['Sub-genres'] = df['Sub-genres'].apply(lambda x: x.replace('[', ''))
df['Sub-genres'] = df['Sub-genres'].apply(lambda x: x.replace(']', ''))
df['Sub-genres'] = df['Sub-genres'].apply(lambda x: x.replace("'", ''))
df['Sub-genres'] = df['Sub-genres'].apply(lambda x: x.replace(', ', ' '))
df['Sub-genres'] = df['Sub-genres'].apply(lambda x: x.split(' '))
df['Top Sub-Genres'] = df['Sub-genres'].apply(lambda x: statistics.multimode(x))

In [6]:
genres = {'latin':'latin',
              'reggaeton': 'latin',
              'latino': 'latin',
              'ritmo': 'latin',
              'salsa': 'latin',
              'mexican': 'latin',
              'alternative': 'indie & alternative',
              'indie': 'indie & alternative',
              'experimental': 'indie & alternative',
              'atl': 'indie & alternative',
              'rock': 'rock & metal',
              'metal': 'rock & metal',
              'punk': 'rock & metal',
              'pop':'pop', 
              'blues':'blues & rhythm',
              'funk': 'blues & rhythm',
              'funky': 'blues & rhythm',
              'jazz': 'jazz',
              'country': 'country', 
              'folk':'folk', 
              'classical':'classical',
              'hip': 'hip hop & rap',
              'hop': 'hip hop & rap',
              'rap': 'hip hop & rap',
              'trap': 'hip hop & rap',
              'drill': 'hip hop & rap',
              'gospel': 'christian & gospel',
              'christian': 'christian & gospel',
              'dance': 'electronic, house & dance',
              'edm' : 'electronic, house & dance',
              'electronic': 'electronic, house & dance',
              'electropop': 'electronic, house & dance',
              'melbourne': 'electronic, house & dance',
              'bounce': 'electronic, house & dance',
              'house': 'electronic, house & dance',
              'grime': 'electronic, house & dance',
              'electro': 'electronic, house & dance',
              'electronica': 'electronic, house & dance',
              'trance': 'electronic, house & dance',
              'r&b': 'r&b & soul',
              'soul': 'r&b & soul',
              'soundtrack': 'soundtrack',
              'hollywood': 'soundtrack',
              'show': 'broadway',
              'tunes': 'broadway',
              'broadway': 'broadway',
              'disney': 'broadway',
              'k-pop': 'k-pop',
              'reggae': 'reggae'
         }

print('No. of Genres', len(set(genres.values())))

def pick_genre(sub_genres, genres=genres):
    
    if sub_genres == ['']:
        return 'unknown'
    
    for element in sub_genres:
        if element in list(genres.keys()):
            return genres[element]
        
    return 'other'
        

No. of Genres 17


In [7]:
df['Genre'] = df['Top Sub-Genres'].apply(pick_genre)
df['Year'] = df['Date'].apply(lambda x: int(x[0:4]))
df.head()

,Date,Track Name,Streams,Position,Artist Name,Artist No.,No. of Artists,Artist Genre,Track URI,Country,Song_days_onchart,Profit,Sub-genres,Top Sub-Genres,Genre,Year
0,2017-01-01,Lighthouse - Andrelli Remix,149929.0,183.0,Hearts & Colors,1,2,['viral pop'],04CttTezSnv71USiiG9mIo,United States,1,496.26499,"[viral, pop]","[viral, pop]",pop,2017
1,2017-01-01,Lighthouse - Andrelli Remix,149929.0,183.0,Andrelli,2,2,['swedish pop'],04CttTezSnv71USiiG9mIo,United States,1,496.26499,"[swedish, pop]","[swedish, pop]",pop,2017
2,2017-01-01,In the Name of Love,435945.0,27.0,Martin Garrix,1,2,"['big room', 'dance pop', 'edm', 'electro hous...",04DwTuZ2VBdJCCC5TROn7L,United States,1,1442.97795,"[big, room, dance, pop, edm, electro, house, p...","[pop, house]",pop,2017
3,2017-01-01,In the Name of Love,435945.0,27.0,Bebe Rexha,2,2,"['dance pop', 'electropop', 'pop', 'pop dance'...",04DwTuZ2VBdJCCC5TROn7L,United States,1,1442.97795,"[dance, pop, electropop, pop, pop, dance, post...",[pop],pop,2017
4,2017-01-01,Party,151474.0,178.0,Chris Brown,1,3,"['dance pop', 'pop', 'pop rap', 'r&b', 'rap']",05Z7jet4VDNVgNQWcYHnrk,United States,1,501.37894,"[dance, pop, pop, pop, rap, r&b, rap]",[pop],pop,2017


In [8]:
def create_edgelist(data):
        
    #Create dictionary of track to genres of genres of artists
    track_genre = {}
    turi_list = list(data['Track URI'])
    g_list = list(data['Genre'])
    
    for i in range(len(data)):
    
        uri = turi_list[i]
        genre = g_list[i]
        
        if uri in track_genre:
            track_genre[uri].append(genre)
        else:
            track_genre[uri] = [genre]
            
    #Create Edgelist dataframee
    track_list = []
    genre1_list = []
    genre2_list = []
   

    for key, values in track_genre.items():
        for i in range(1, len(values)):
            track_list.append(key)
            genre1_list.append(values[0])
            genre2_list.append(values[i])
            
    
    edge_list = pd.DataFrame({'Track URI': track_list,
                              'Artist Genre': genre1_list, 
                              'Collaborator Genre': genre2_list})



    return edge_list
    

In [12]:
def generate_network_plotly(data, year=[],  genres = [], metric='Count'):
    
    #Reduce dataframe
    if year != []:
        reduced_data = data[data['Year'].isin(year)][['Track URI', 'Artist Name', 'Genre']].drop_duplicates(ignore_index=True)
    else:
        reduced_data = data[['Track URI', 'Artist Name', 'Genre']].drop_duplicates(ignore_index=True)
    
    #Create Graph
    digraph = nx.DiGraph()
    
    #Create Node List
    nodes_df = reduced_data.groupby(['Genre'])[['Artist Name']].count().reset_index(drop=False).rename(columns={'Artist Name': 'Count'})
    nodes_df['Size'] = nodes_df['Count'].apply(lambda x: 5*x**(1/3))
    nodes_df = nodes_df[nodes_df.Genre != 'unknown']
    nodes_df = nodes_df[nodes_df.Genre != 'other'].reset_index(drop=False)
    
    #Add Nodes
    for i in range(len(nodes_df)):
        digraph.add_node(nodes_df.Genre[i], title=nodes_df.Genre[i], count=nodes_df.Count[i], size=nodes_df.Size[i])
        
    #Create Edge List
    edges_df = create_edgelist(reduced_data)
    
    edges_df = edges_df[edges_df['Artist Genre'] != edges_df['Collaborator Genre']]
    
    for g in ['unknown', 'other']:
        edges_df = edges_df[edges_df['Artist Genre'] != g]
        edges_df = edges_df[edges_df['Collaborator Genre'] != g]
    
    if genres != []:
        a, b = edges_df[edges_df['Artist Genre'].isin(genres)], edges_df[edges_df['Collaborator Genre'].isin(genres)]
        edges_df = pd.concat([a,b])
        
    if metric == 'Count':
        edge_list_reduced = edges_df.groupby(['Artist Genre', 'Collaborator Genre'])[['Artist Genre', 'Collaborator Genre']].count().rename(columns={'Artist Genre': 'Count', 'Collaborator Genre':'Weight'}).reset_index(drop=False)
        edge_list_reduced['Weight'] = edge_list_reduced['Count'].apply(lambda x: x/(max(edge_list_reduced['Count'])))
        edge_list_reduced['Color'] =  edge_list_reduced['Weight'].apply(lambda x: plt.cm.Wistia(x))
        title_label = 'No. of Collaborations'
        
    elif metric == 'Streams':
        track_streams = data.groupby(['Track URI'])[['Streams']].sum().reset_index(drop=False)
        edge_list_reduced = edges_df.set_index('Track URI').join(track_streams.set_index('Track URI'))
        edge_list_reduced = edge_list_reduced.groupby(['Artist Genre', 'Collaborator Genre'])[['Streams']].sum().reset_index(drop=False)
        edge_list_reduced['Weight'] = edge_list_reduced['Streams'].apply(lambda x: x / edge_list_reduced['Streams'].max())
        edge_list_reduced['Color'] =  edge_list_reduced['Weight'].apply(lambda x: plt.cm.Wistia(x))
        title_label = 'Cumulative No. of Streams'
        
        
    #Add Edges
    for i in range(len(edge_list_reduced)):
        u = edge_list_reduced['Artist Genre'][i]
        v = edge_list_reduced['Collaborator Genre'][i]
        color = edge_list_reduced['Color'][i]
        m = edge_list_reduced[metric][i]
        digraph.add_edge(u, v, color=color, metric = m)
        
    
    #Plot Graph
    
    pos = nx.circular_layout(digraph)
    
    for node in digraph.nodes:
        digraph.nodes[node]['pos'] = list(pos[node])


    edge_x = []
    edge_y = []
    edge_weight = []
    for edge in digraph.edges():
        x0, y0 = digraph.nodes[edge[0]]['pos']
        x1, y1 = digraph.nodes[edge[1]]['pos']
        weight = digraph.edges[edge]['metric']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_weight.append(weight)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(
            width=2,
        ),
        hoverinfo='none',
        mode='lines',
        opacity = 0.5
    )
        
    node_x = []
    node_y = []
    node_size = []
    node_label = []
    hover_label = []
    for node in digraph.nodes():
        x, y = digraph.nodes[node]['pos']
        size = digraph.nodes[node]['size']
        n_label = '<b>' + digraph.nodes[node]['title'] + '</b>'
        h_label = str(digraph.nodes[node]['count']) + ' artists'
        node_x.append(x)
        node_y.append(y)
        node_size.append(size)
        node_label.append(n_label)
        hover_label.append(h_label)
    
        
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        hovertext= hover_label,
        text = node_label,
        textfont_color = 'white',
        marker=dict(
            showscale=False,
            color='#76984e',
            size=node_size,
            line_width=0,
            opacity=1,
        ))
        
    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title=title_label,
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                plot_bgcolor='rgb(0,0,0)',
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                annotations = [ dict(
                    ax=(digraph.nodes[edge[0]]['pos'][0] + digraph.nodes[edge[1]]['pos'][0]) / 2,
                    ay=(digraph.nodes[edge[0]]['pos'][1] + digraph.nodes[edge[1]]['pos'][1]) / 2, axref='x', ayref='y',
                    x=(digraph.nodes[edge[1]]['pos'][0] * 3 + digraph.nodes[edge[0]]['pos'][0]) / 4,
                    y=(digraph.nodes[edge[1]]['pos'][1] * 3 + digraph.nodes[edge[0]]['pos'][1]) / 4, xref='x', yref='y',
                    showarrow=True,
                    arrowhead=2,
                    arrowsize=2,
                    arrowwidth=1,
                    opacity=1,
                    arrowcolor = mpl.colors.to_hex(digraph.edges[edge]['color']),
                    )
                for edge in digraph.edges]
             ))
    
    
    #Add custom color bar
    wistia_color = [mpl.colors.to_hex(plt.cm.Wistia(i/10)) for i in range(0, 12, 2)]
    
    mx_value = int(edge_list_reduced[metric].max())
    
    if mx_value % 10 == 0:
        colorbar_max = mx_value
    else:
        colorbar_max = mx_value + (10 - mx_value % 10)
        
    interval = colorbar_max // 5
    tick_values = np.arange(0, colorbar_max + interval, interval)        
    
    colorbar_trace = go.Scatter(x=[None], y=[None],
                                mode='markers',
                                marker=dict(
                                    colorscale=wistia_color,
                                    showscale=True,
                                    cmin=0,
                                    cmax=colorbar_max,
                                    colorbar=dict(
                                        thickness=15,
                                        tickvals=tick_values,
                                        tickformat = '0.2s',
                                        outlinewidth=0)
                             ),
                             hoverinfo='none'
                            )
    
    fig.add_trace(colorbar_trace)
    
    warnings.filterwarnings('ignore')
    fig.show()

In [13]:
generate_network_plotly(df)